In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 24, 8
%matplotlib inline

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.utils import resample

In [2]:
def gini(solution, submission):
    df = zip(solution, submission, range(len(solution)))
    df = sorted(df, key=lambda x: (x[1],-x[2]), reverse=True)
    rand = [float(i+1)/float(len(df)) for i in range(len(df))]
    totalPos = float(sum([x[0] for x in df]))
    cumPosFound = [df[0][0]]
    for i in range(1,len(df)):
        cumPosFound.append(cumPosFound[len(cumPosFound)-1] + df[i][0])
    Lorentz = [float(x)/totalPos for x in cumPosFound]
    Gini = [Lorentz[i]-rand[i] for i in range(len(df))]
    return sum(Gini)

def normalized_gini(solution, submission):
    normalized_gini = gini(solution, submission)/gini(solution, solution)
    return normalized_gini

In [3]:
train_df = pd.read_csv('../data/train2.csv', na_values="999") # .iloc[0:200,:]
test_df = pd.read_csv('../data/test2.csv', na_values="999")

X = train_df
X = X.drop(['target'], axis=1)
y = train_df['target']

X = X.fillna(999)
test_df = test_df.fillna(999)
test_df = test_df.drop(['id'], axis=1)

X = X.values
y = y.values

In [7]:
clf = GradientBoostingClassifier(presort=True)

In [12]:
param_grid = { "loss":['deviance'],
               "n_estimators":[10,20,30,40,50,55,70,80,100],
               'max_features': [2,3,4,5,6,7,'auto','sqrt','log2',None],
               "max_depth" : [i for i in range(1,10,1)],
               'min_samples_split': [.1,.13,0.25,0.3,0.45],
               'max_leaf_nodes': [2,3,4,5,6,7,8,9,10,None],
               "min_samples_leaf" : [i for i in range(1,10,1)],
               "subsample":[i/100.0 for i in range(80,100)],
               'min_impurity_decrease':[i/10.for i in range(1,10)]
               }

n_iter_search = 800
gini_scorer = make_scorer(normalized_gini, greater_is_better = True)
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_grid,
                                   scoring = gini_scorer,
                                   n_iter=n_iter_search,
                                   n_jobs=-1,
                                   cv= StratifiedKFold(n_splits=5, shuffle=True),
                                   verbose=2)
random_search.fit(X, y)

Fitting 5 folds for each of 800 candidates, totalling 4000 fits
[CV] subsample=0.92, n_estimators=80, min_samples_split=0.3, min_samples_leaf=8, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=7, max_depth=4, loss=deviance 
[CV] subsample=0.92, n_estimators=80, min_samples_split=0.3, min_samples_leaf=8, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=7, max_depth=4, loss=deviance 
[CV] subsample=0.92, n_estimators=80, min_samples_split=0.3, min_samples_leaf=8, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=7, max_depth=4, loss=deviance 
[CV] subsample=0.92, n_estimators=80, min_samples_split=0.3, min_samples_leaf=8, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=7, max_depth=4, loss=deviance 
[CV] subsample=0.92, n_estimators=80, min_samples_split=0.3, min_samples_leaf=8, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=7, max_depth=4, loss=deviance 
[CV] subsample=0.81, n_estimators=80, min_samples_split=0.1, min_samples_leaf=4, min_i

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.3min


[CV]  subsample=0.89, n_estimators=55, min_samples_split=0.1, min_samples_leaf=8, min_impurity_decrease=0.9, max_leaf_nodes=4, max_features=sqrt, max_depth=8, loss=deviance, total=  55.8s
[CV] subsample=0.94, n_estimators=40, min_samples_split=0.1, min_samples_leaf=5, min_impurity_decrease=0.3, max_leaf_nodes=7, max_features=sqrt, max_depth=1, loss=deviance 
[CV]  subsample=0.89, n_estimators=55, min_samples_split=0.1, min_samples_leaf=8, min_impurity_decrease=0.9, max_leaf_nodes=4, max_features=sqrt, max_depth=8, loss=deviance, total=  57.6s
[CV] subsample=0.96, n_estimators=20, min_samples_split=0.13, min_samples_leaf=5, min_impurity_decrease=0.5, max_leaf_nodes=9, max_features=2, max_depth=1, loss=deviance 
[CV]  subsample=0.89, n_estimators=55, min_samples_split=0.1, min_samples_leaf=8, min_impurity_decrease=0.9, max_leaf_nodes=4, max_features=sqrt, max_depth=8, loss=deviance, total=  56.0s
[CV] subsample=0.96, n_estimators=20, min_samples_split=0.13, min_samples_leaf=5, min_impuri

[CV]  subsample=0.98, n_estimators=10, min_samples_split=0.3, min_samples_leaf=8, min_impurity_decrease=0.8, max_leaf_nodes=7, max_features=auto, max_depth=6, loss=deviance, total= 1.1min
[CV] subsample=0.89, n_estimators=10, min_samples_split=0.1, min_samples_leaf=1, min_impurity_decrease=0.6, max_leaf_nodes=None, max_features=5, max_depth=7, loss=deviance 
[CV]  subsample=0.98, n_estimators=10, min_samples_split=0.3, min_samples_leaf=8, min_impurity_decrease=0.8, max_leaf_nodes=7, max_features=auto, max_depth=6, loss=deviance, total= 1.1min
[CV] subsample=0.89, n_estimators=10, min_samples_split=0.1, min_samples_leaf=1, min_impurity_decrease=0.6, max_leaf_nodes=None, max_features=5, max_depth=7, loss=deviance 
[CV]  subsample=0.89, n_estimators=10, min_samples_split=0.1, min_samples_leaf=1, min_impurity_decrease=0.6, max_leaf_nodes=None, max_features=5, max_depth=7, loss=deviance, total=  19.3s
[CV] subsample=0.89, n_estimators=10, min_samples_split=0.1, min_samples_leaf=1, min_impur

[CV]  subsample=0.91, n_estimators=55, min_samples_split=0.1, min_samples_leaf=1, min_impurity_decrease=0.6, max_leaf_nodes=None, max_features=auto, max_depth=7, loss=deviance, total= 7.2min
[CV]  subsample=0.91, n_estimators=55, min_samples_split=0.1, min_samples_leaf=1, min_impurity_decrease=0.6, max_leaf_nodes=None, max_features=auto, max_depth=7, loss=deviance, total= 7.2min
[CV] subsample=0.83, n_estimators=80, min_samples_split=0.45, min_samples_leaf=7, min_impurity_decrease=0.9, max_leaf_nodes=2, max_features=auto, max_depth=8, loss=deviance 
[CV] subsample=0.83, n_estimators=80, min_samples_split=0.45, min_samples_leaf=7, min_impurity_decrease=0.9, max_leaf_nodes=2, max_features=auto, max_depth=8, loss=deviance 
[CV]  subsample=0.84, n_estimators=55, min_samples_split=0.1, min_samples_leaf=9, min_impurity_decrease=0.2, max_leaf_nodes=7, max_features=auto, max_depth=2, loss=deviance, total= 4.4min
[CV] subsample=0.83, n_estimators=80, min_samples_split=0.45, min_samples_leaf=7, 

...

[CV]  subsample=0.95, n_estimators=40, min_samples_split=0.1, min_samples_leaf=3, min_impurity_decrease=0.2, max_leaf_nodes=8, max_features=4, max_depth=7, loss=deviance, total=  44.9s
[CV] subsample=0.81, n_estimators=10, min_samples_split=0.1, min_samples_leaf=4, min_impurity_decrease=0.7, max_leaf_nodes=9, max_features=auto, max_depth=5, loss=deviance 
[CV]  subsample=0.95, n_estimators=40, min_samples_split=0.1, min_samples_leaf=3, min_impurity_decrease=0.2, max_leaf_nodes=8, max_features=4, max_depth=7, loss=deviance, total=  47.0s
[CV] subsample=0.92, n_estimators=55, min_samples_split=0.3, min_samples_leaf=2, min_impurity_decrease=0.8, max_leaf_nodes=4, max_features=log2, max_depth=5, loss=deviance 
[CV]  subsample=0.95, n_estimators=40, min_samples_split=0.1, min_samples_leaf=3, min_impurity_decrease=0.2, max_leaf_nodes=8, max_features=4, max_depth=7, loss=deviance, total=  45.6s
[CV] subsample=0.92, n_estimators=55, min_samples_split=0.3, min_samples_leaf=2, min_impurity_decre

[CV]  subsample=0.91, n_estimators=50, min_samples_split=0.13, min_samples_leaf=1, min_impurity_decrease=0.8, max_leaf_nodes=5, max_features=None, max_depth=9, loss=deviance, total= 5.2min
[CV] subsample=0.87, n_estimators=50, min_samples_split=0.45, min_samples_leaf=5, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=None, max_depth=4, loss=deviance 
[CV]  subsample=0.91, n_estimators=50, min_samples_split=0.13, min_samples_leaf=1, min_impurity_decrease=0.8, max_leaf_nodes=5, max_features=None, max_depth=9, loss=deviance, total= 5.2min
[CV] subsample=0.87, n_estimators=50, min_samples_split=0.45, min_samples_leaf=5, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=None, max_depth=4, loss=deviance 
[CV]  subsample=0.91, n_estimators=50, min_samples_split=0.13, min_samples_leaf=1, min_impurity_decrease=0.8, max_leaf_nodes=5, max_features=None, max_depth=9, loss=deviance, total= 5.2min
[CV] subsample=0.87, n_estimators=50, min_samples_split=0.45, min_samples_leaf=5, min

[CV]  subsample=0.87, n_estimators=50, min_samples_split=0.45, min_samples_leaf=5, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=None, max_depth=4, loss=deviance, total= 3.2min
[CV]  subsample=0.87, n_estimators=50, min_samples_split=0.45, min_samples_leaf=5, min_impurity_decrease=0.5, max_leaf_nodes=6, max_features=None, max_depth=4, loss=deviance, total= 2.9min


[Parallel(n_jobs=-1)]: Done 4000 out of 4000 | elapsed: 480.3min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
          error_score='raise',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100, presort=True,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
          fit_params=None, iid=True, n_iter=800, n_jobs=-1,
          param_distributions={'loss': ['deviance'], 'n_estimators': [10, 20, 30, 40, 50, 55, 70, 80, 100], 'max_features': [2, 3, 4, 5, 6, 7, 'auto', 'sqrt', 'log2', None], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'min_samples_split': [0.1, 0.13, 0.25, 0.3, 0.45], 'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, ...95, 0.96, 0.97, 0.98, 

In [17]:
random_search.best_score_

0.0011483890749521756

In [16]:
random_search.cv_results_['mean_test_score']

array([ 0.00058638,  0.0006777 ,  0.00113998,  0.00059347,  0.00073394,
        0.00066177,  0.00059347,  0.00072672,  0.00059347,  0.00059347,
        0.00059347,  0.00059347,  0.00059347,  0.00106597,  0.0005934 ,
        0.00059347,  0.00059347,  0.00058933,  0.00059347,  0.00059347,
        0.00066023,  0.00084642,  0.00059347,  0.00084185,  0.00059347,
        0.00059207,  0.00059047,  0.00073282,  0.00059347,  0.00059347,
        0.00059347,  0.00059347,  0.00059347,  0.00059347,  0.00059347,
        0.00059347,  0.00059347,  0.00059347,  0.00062468,  0.00059347,
        0.00059347,  0.00073378,  0.00059347,  0.00059347,  0.00068928,
        0.00059347,  0.00059347,  0.00059347,  0.00059347,  0.00059347,
        0.00059347,  0.00059347,  0.00059347,  0.00059347,  0.00066387,
        0.00059347,  0.00059347,  0.00059967,  0.00059347,  0.00067719,
        0.00059347,  0.00059347,  0.00059347,  0.00059347,  0.00059347,
        0.00065453,  0.00059347,  0.0006555 ,  0.00059347,  0.00